<a href="https://colab.research.google.com/github/Raaj-4320/the-exam-paper-generator/blob/main/que_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00


In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [110]:
topic = "youtube"

In [111]:
def search_wikipedia(topic):
    search_url = f"https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch={topic}&format=json"
    response = requests.get(search_url)
    search_results = response.json()
    return search_results['query']['search']

def clean_snippet(snippet):
    soup = BeautifulSoup(snippet, "html.parser")
    text = soup.get_text()
    return text

def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text)

    # Convert to lowercase
    tokens = [word.lower() for word in tokens]

    # Remove stop words and non-alphabetic tokens
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]

    return ' '.join(tokens)

search_results = search_wikipedia(topic)

# Extract titles from the search results
titles = [result['title'] for result in search_results]

# Print related topics (titles)
all_topics = []
print("Related topics:")
for title in titles:
  all_topics.append(title)

print(all_topics)

Related topics:
['YouTube', 'YouTube Music', 'YouTube Premium', 'YouTube Shorts', 'YouTube (YouTube channel)', 'Youtube-dl', 'List of most-viewed YouTube videos', 'History of YouTube', 'List of most-subscribed YouTube channels', 'YouTube poop']


In [112]:
import requests

def get_wikipedia_extract(title, length="short"):
    # Define the endpoint
    endpoint = "https://en.wikipedia.org/w/api.php"

    # Set the extract length parameters
    if length == "short":
        exchars = 500  # Limit to approximately one paragraph
        params = {
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            "exchars": exchars,
            "explaintext": True
        }
    elif length == "medium":
        exsentences = 5  # Limit to approximately two paragraphs
        params = {
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            "exsentences": exsentences,
            "explaintext": True
        }
    else:
        params = {
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            "exintro": True,
            "explaintext": True
        }

    # Make the request to the Wikipedia API
    response = requests.get(endpoint, params=params)
    data = response.json()

    # Extract the page information
    pages = data['query']['pages']
    page = next(iter(pages.values()))  # Get the first (and only) page

    # Extract the extract text
    extract = page.get("extract", "No extract found.")

    return extract

# Dictionary to store the extracts
corpus = {}

# Iterate through each title and get a paragraph (full extract) for each
for title in titles:
    extract = get_wikipedia_extract(title, length="full")
    print(f"Title: {title}\nExtract:\n{extract}\n")
    corpus[title] = extract

# Now 'corpus' contains all the extracts with titles as keys
print("Corpus dictionary:", corpus)


Title: YouTube
Extract:
YouTube is an American online video sharing platform owned by Google. Accessible worldwide, YouTube was launched on February 14, 2005, by Steve Chen, Chad Hurley, and Jawed Karim, three former employees of PayPal. Headquartered in San Bruno, California, United States, it is the second most visited website in the world, after Google Search. As of January 2024, YouTube has more than 2.7 billion monthly users, who collectively watch more than one billion hours of videos every day. As of May 2019, videos were being uploaded to the platform at a rate of more than 500 hours of content per minute, and as of 2021, there were approximately 14 billion videos in total.
In October 2006, YouTube was purchased by Google for $1.65 billion (equivalent to $2.31 billion in 2023). Google expanded YouTube's business model of generating revenue from advertisements alone, to offering paid content such as movies and exclusive content produced by and for YouTube. It also offers YouTube

In [113]:
from openai import OpenAI

client = OpenAI(
    api_key=''
)

In [114]:
completion = client.chat.completions.create(
    model='gpt-3.5-turbo',
    response_format={ "type": "json_object" },
    messages=[
        {'role':'system','content':'you are a good teacher who sets a good exam papers'},
        {'role': 'user','content':f'create 20 multiple choice questions of this given {extract} with complexity, and mention all answers the with the questions in json format. Please do not explain the ansnwer, just return whatever is written in the options and the exact answer not more not less.'}
        ]
)

In [123]:
qa_data = completion.choices[0].message.content

In [125]:

# Load JSON data into a Python dictionary
data = json.loads(qa_data)

# Create a dictionary with the desired format
cleaned_data = [
    {
        "question": item['question'],
        "choices": item['options'],
        "correct_answer": item['answer']
    }
    for item in data['questions']
]


In [149]:
for i in range(len(cleaned_data)):
    print("Question", i + 1)
    print(cleaned_data[i]['question'])

    # Print each choice with its label
    choices_dict = cleaned_data[i]['choices']
    for label, choice in choices_dict.items():
        print(f"{label} : {choice}")

    # Find the correct answer label
    correct_answer = cleaned_data[i]['correct_answer']
    print("Correct Answer:",correct_answer,"\n")


Question 1
What is a YouTube poop?
A : A type of video mashup
B : A type of sandwich
C : A type of dance
D : A type of painting
Correct Answer: A 

Question 2
Where are YouTube poops traditionally uploaded?
A : Facebook
B : Instagram
C : YouTube
D : Twitter
Correct Answer: C 

Question 3
What is the main purpose of creating YouTube poops?
A : Educational
B : Informative
C : Humorous
D : Sad
Correct Answer: C 

Question 4
Which of the following is NOT a characteristic of YouTube poops?
A : Vulgar
B : Educational
C : Profane
D : Satirical
Correct Answer: B 

Question 5
What kind of sources are typically used to create YouTube poops?
A : Newly created media
B : Pre-existing media
C : Live performances
D : Video games
Correct Answer: B 

Question 6
Which of the following is a common theme in YouTube poops?
A : Seriousness
B : Compassion
C : Absurdity
D : Peace
Correct Answer: C 

Question 7
What website is often used for sharing YouTube poops?
A : Vimeo
B : YouTube
C : Netflix
D : Hulu
Cor